In [12]:
!pip install pyworld inflect tgt einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
  Created wheel for pyworld: filename=pyworld-0.3.4-cp310-cp310-linux_x86_64.whl size=202814 sha256=54b6e26e582a61a324624d8227ab91d4a326f99d66f12329396e3bbfdf354c6c
  Stored in directory: /root/.cache/pip/wheels/66/09/8a/a1d79b73d59756f66e9bfe55a199840efc7473adb76ddacdfd
  Created wheel for tgt: filename=tgt-1.4.4-py3-none-any.whl size=28929 sha256=532ad1c6265875ce5845b4ef62b943013e501f774b214e1327c0d569021660ba
  Stored in directory: /root/.cache/pip/wheels/09/e6/aa/821531faeb4e05a65d1c763570e90791467cf0c3f1622dc7e2
Successfully built pyworld tgt


In [13]:
## Standard libraries
import os
import numpy as np
import pandas as pd 
import random
import math
import json
from functools import partial
from PIL import Image
import wandb
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
from pytorch_lightning.loggers import WandbLogger
wandb_api_key = '430e8c7ef92cf79a3d7c3d02e3d961257153181f'
os.environ["WANDB_API_KEY"] = wandb_api_key

import sys; sys.path.insert(0, '/..')
os.chdir(os.path.join(os.getcwd(), 'masters-final'))


from text import _clean_text
from text import _symbol_to_id
from audio.tools import inv_mel_spec
from audio.stft import TacotronSTFT
from collections import OrderedDict

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [10]:
# import os
# os.listdir('./masters-final/text')

['ipa.py',
 'pinyin.py',
 'symbols.py',
 'cmudict.py',
 'cleaners.py',
 'numbers.py',
 '__init__.py']

In [9]:
# # Run the following if it's the first time running in this kernel
#!git clone https://github.com/SitholeDavid/masters-final
!git pull

Cloning into 'masters-final'...
remote: Enumerating objects: 24849, done.
remote: Counting objects: 100% (24200/24200), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 24849 (delta 24064), reused 24142 (delta 24022), pack-reused 649
Receiving objects: 100% (24849/24849), 332.24 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (24225/24225), done.
Updating files: 100% (24202/24202), done.


In [4]:
# import librosa.display
# mel_spec = torch.from_numpy(np.load('./preprocessed_data/LJSpeech/mel/LJSpeech-mel-LJ048-0114.npy'))
# # mel_spec = torch.from_numpy(np.load('/kaggle/working/masters-final/preprocessed_data/SpanishSingleSpeaker/mel/SpanishSingleSpeaker-mel-19demarzo_0022.npy'))
# librosa.display.specshow(mel_spec.T.numpy())
# stft = TacotronSTFT(filter_length=1024, hop_length=256, win_length=1024, n_mel_channels=80, sampling_rate=22050, mel_fmin=0, mel_fmax=8000)
# inv_mel_spec(mel_spec.T, 'file.wav', stft)

### Prepare Spanish Dataset

In [14]:
# !python3 prepare_align.py config/SpanishSingleSpeaker/preprocess.yaml

100%|█████████████████████████████████████| 11111/11111 [08:37<00:00, 21.48it/s]


In [15]:
# !python3 preprocess.py config/SpanishSingleSpeaker/preprocess.yaml

Processing Data ...
100%|███████████████████████████████████████████| 1/1 [19:59<00:00, 1199.53s/it]
Computing statistic quantities ...
Total time: 17.2391490047871 hours


### Dataset and Loaders

In [16]:
!python3 prepare_align.py config/LJSpeech/preprocess.yaml

13100it [08:36, 25.39it/s]


In [17]:
!python3 preprocess.py config/LJSpeech/preprocess.yaml

Processing Data ...
100%|███████████████████████████████████████████| 1/1 [21:56<00:00, 1316.25s/it]
Computing statistic quantities ...
Total time: 23.62337556059461 hours


In [7]:
# class SpanishDataset(Dataset):
#     def __init__(self,  input_dir, input_file, max_src_len, max_trg_len):
#         self.input_dir = input_dir
#         self.max_src_len = max_src_len
#         self.max_trg_len = max_trg_len
        
#         files = pd.read_csv(os.path.join(input_dir, input_file), sep='|', header=None)
#         files.columns = ['file', 'speaker', 'phones', 'text'] 
        
#         # only consider valid phoneme sequences
#         for index, row in tqdm(files.iterrows(), total=len(files)):
#             if not all(symbol in _symbol_to_id for symbol in self._process_phones(row['phones'])):
#                 files.drop(index, inplace=True)

#         self.file_names = files

#     def _process_phones(self, phones):
#         phones = phones.replace('{', '').replace('}', '').strip().split(' ')
#         mapped_phones = [f'@{phone}' for phone in phones]
#         return mapped_phones
        
#     def __len__(self):
#         return len(self.file_names)

#     def __getitem__(self, idx):
#         file_name = self.file_names.iloc[idx]['file']
        
#         mel = np.load(os.path.join(self.input_dir, 'mel', f'SpanishSingleSpeaker-mel-{file_name}.npy'))
#         duration = np.load(os.path.join(self.input_dir, 'duration', f'SpanishSingleSpeaker-duration-{file_name}.npy'))
#         energy = np.load(os.path.join(self.input_dir, 'energy', f'SpanishSingleSpeaker-energy-{file_name}.npy'))
#         pitch = np.load(os.path.join(self.input_dir, 'pitch', f'SpanishSingleSpeaker-pitch-{file_name}.npy'))
#         phones = self._process_phones(self.file_names.iloc[idx]['phones'])
        
#         phone_mapping = torch.tensor([ _symbol_to_id[symbol] for symbol in phones ])

#         src_len = torch.tensor(len(phones))
#         trg_len = torch.tensor(mel.shape[0])

#         phoneme_pad_length = self.max_src_len - src_len
#         mel_pad_length = self.max_trg_len - trg_len
        
#         phone_mapping = F.pad(phone_mapping, (0, phoneme_pad_length), mode='constant', value=0)
#         duration = F.pad( torch.tensor(duration), (0, phoneme_pad_length), mode='constant', value=0)
#         energy = F.pad( torch.tensor(energy), (0, phoneme_pad_length), mode='constant', value=0)
#         pitch = F.pad( torch.tensor(pitch), (0, phoneme_pad_length), mode='constant', value=0)
#         mel = F.pad(torch.tensor(mel), (0, 0, 0, mel_pad_length), mode='constant', value=0)
#         return  phone_mapping, src_len, mel, trg_len, duration, energy, pitch
    
# train_dataset = SpanishDataset('./preprocessed_data/SpanishSingleSpeaker', 'train.txt', max_src_len=200, max_trg_len=1000)
# val_dataset = SpanishDataset('./preprocessed_data/SpanishSingleSpeaker', 'val.txt', max_src_len=200, max_trg_len=1000)

# # set shuffle to false since we already shuffle when the data is split into train/test
# train_loader = DataLoader(train_dataset, shuffle=False, batch_size=4, pin_memory=True)
# val_loader = DataLoader(val_dataset, shuffle=False, batch_size=4)

  0%|          | 0/10437 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [18]:
class LJSpeechDataset(Dataset):
    def __init__(self,  input_dir, input_file, max_src_len, max_trg_len):
        self.input_dir = input_dir
        self.max_src_len = max_src_len
        self.max_trg_len = max_trg_len
        
        files = pd.read_csv(os.path.join(input_dir, input_file), sep='|', header=None)
        files.columns = ['file', 'speaker', 'phones', 'text'] 
        
        # only consider valid phoneme sequences
        for index, row in tqdm(files.iterrows(), total=len(files)):
            if not all(symbol in _symbol_to_id for symbol in self._process_phones(row['phones'])):
                files.drop(index, inplace=True)

        self.file_names = files

    def _process_phones(self, phones):
        phones = phones.replace('{', '').replace('}', '').strip().split(' ')
        mapped_phones = [f'@{phone}' if str.isalnum(phone) else phone for phone in phones]
        return mapped_phones
        
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names.iloc[idx]['file']
        
        mel = np.load(os.path.join(self.input_dir, 'mel', f'LJSpeech-mel-{file_name}.npy'))
        duration = np.load(os.path.join(self.input_dir, 'duration', f'LJSpeech-duration-{file_name}.npy'))
        energy = np.load(os.path.join(self.input_dir, 'energy', f'LJSpeech-energy-{file_name}.npy'))
        pitch = np.load(os.path.join(self.input_dir, 'pitch', f'LJSpeech-pitch-{file_name}.npy'))
        phones = self._process_phones(self.file_names.iloc[idx]['phones'])
        
        phone_mapping = torch.tensor([ _symbol_to_id[symbol] for symbol in phones ])

        src_len = torch.tensor(len(phones))
        trg_len = torch.tensor(mel.shape[0])

        phoneme_pad_length = self.max_src_len - src_len
        mel_pad_length = self.max_trg_len - trg_len
        
        phone_mapping = F.pad(phone_mapping, (0, phoneme_pad_length), mode='constant', value=0)
        duration = F.pad( torch.tensor(duration), (0, phoneme_pad_length), mode='constant', value=0)
        energy = F.pad( torch.tensor(energy), (0, phoneme_pad_length), mode='constant', value=0)
        pitch = F.pad( torch.tensor(pitch), (0, phoneme_pad_length), mode='constant', value=0)
        mel = F.pad(torch.tensor(mel), (0, 0, 0, mel_pad_length), mode='constant', value=0)
        return  phone_mapping, src_len, mel, trg_len, duration, energy, pitch
    
train_dataset = LJSpeechDataset('./preprocessed_data/LJSpeech', 'train.txt', max_src_len=200, max_trg_len=1000)
val_dataset = LJSpeechDataset('./preprocessed_data/LJSpeech', 'val.txt', max_src_len=200, max_trg_len=1000)

# set shuffle to false since we already shuffle when the data is split into train/test
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=4, pin_memory=True)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=4)

  0%|          | 0/12273 [00:00<?, ?it/s]

  0%|          | 0/454 [00:00<?, ?it/s]

#### Model Layers

In [19]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [20]:
stft = TacotronSTFT(filter_length=1024, hop_length=256, win_length=1024, n_mel_channels=80, sampling_rate=22050, mel_fmin=0, mel_fmax=8000)
#inv_mel_spec(mel_spec.T, 'file.wav', stft)

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout, hidden_dim, kernel_size):
        super().__init__()
        
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout, batch_first=True)
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.mlp = nn.Sequential(
            nn.LayerNorm(embed_dim),
            Rearrange('B S E -> B E S'),
            nn.Conv1d(embed_dim, hidden_dim, kernel_size, padding = (kernel_size - 1) // 2),
            #nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Conv1d(hidden_dim, embed_dim, kernel_size, padding = (kernel_size - 1) // 2),
            #nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout),
            Rearrange('B E S-> B S E')
        )
        
        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x, mask=None, fill_mask=None):
        x.masked_fill_(fill_mask, 0.0)
        attn_in = self.layer_norm(x)
        attn_out, _ = self.attention(attn_in, attn_in, attn_in, key_padding_mask=mask)
        x_out = attn_out + x
        
        x_out.masked_fill_(fill_mask, 0.0)
        mlp_out = self.mlp(x_out)
        out = mlp_out + x_out
        out.masked_fill_(fill_mask, 0.0)
        return out
    
    
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, num_layers, kernel_size, max_src_seq_len, src_vocab_size, dropout=0.0):
        super().__init__()
        
        self.pos_embedding = PositionalEncoding(embed_dim, max_src_seq_len)
        self.phone_embedding = nn.Embedding(src_vocab_size, embed_dim, padding_idx=0)
        self.layers = nn.Sequential(*[TransformerBlock(embed_dim, num_heads, dropout, hidden_dim, kernel_size)  for _ in range(num_layers)])
        
    def forward(self, x, mask=None, fill_mask=None):
        x = self.phone_embedding(x)
        x = self.pos_embedding(x)
        
        for l in self.layers:
            x = l(x, mask, fill_mask)
        
        return x
        
class TransformerDecoder(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, num_layers, kernel_size, max_trg_seq_len, dropout=0.0):
        super().__init__()
        
        self.pos_embedding = PositionalEncoding(embed_dim, max_trg_seq_len)
        self.layers = nn.Sequential(*[TransformerBlock(embed_dim, num_heads, dropout, hidden_dim, kernel_size)  for _ in range(num_layers)])
        
    def forward(self, x, mask=None, fill_mask=None):
        x = self.pos_embedding(x)
        
        for l in self.layers:
            x = l(x, mask, fill_mask)
        
        return x
        
class Hidden2Mel(nn.Module):
    def __init__(self, embed_dim, n_mels):
        super().__init__()
        self.layers = nn.Linear(embed_dim, n_mels)
        
    def forward(self, x):
        return self.layers(x)
    
class LengthRegulator(nn.Module):
    def __init__(self, max_trg_len=1000):
        super().__init__()

        self.max_trg_len = max_trg_len

    def forward(self, encoder_output, variance):
        B = encoder_output.shape[0]
        mels = list()

        for b_idx in range(B):
            expanded_seq = torch.concat([ encoder_output[b_idx,i,:].expand(v, -1) for i, v in enumerate(variance[b_idx, :]) ], dim=0)
            seq_len = expanded_seq.shape[0]
            pad_len = self.max_trg_len - seq_len

            if pad_len < 0:
                padded_seq = expanded_seq[:self.max_trg_len, :]
            elif pad_len > 0:
                padded_seq = F.pad( expanded_seq, (0, 0, 0, pad_len), "constant", -1 )

            mels.append(padded_seq)
        expanded_batch = torch.stack(mels, dim=0)
        return expanded_batch
    
class FastSpeechLoss(nn.Module):
    def __init__(self, encoder_max_seq_len, decoder_max_seq_len, mel_channels=80):
        super().__init__()

        self.mel_channels = mel_channels
        self.encoder_max_seq_len = encoder_max_seq_len
        self.decoder_max_seq_len = decoder_max_seq_len
        self.mae_loss = nn.L1Loss()
        self.mse_loss = nn.MSELoss()
   
    def forward(self, 
                h2m_predictions,
                mel_targets,
                p_predictions,
                p_targets,
                e_predictions,
                e_targets,
                log_d_predictions,
                d_targets,
                src_masks,
                trg_masks
               ):
        
        src_masks = ~src_masks
        trg_masks = ~trg_masks
        trg_masks = trg_masks[:, :, :self.mel_channels]
        
        d_targets.requires_grad = False
        p_targets.requires_grad = False
        e_targets.requires_grad = False
        mel_targets.requires_grad = False
        
        log_d_targets = torch.log(d_targets.float() + 1)
        
        p_loss = self.mse_loss(p_predictions.masked_select(src_masks).float(), p_targets.masked_select(src_masks).float())
        e_loss = self.mse_loss(e_predictions.masked_select(src_masks).float(), e_targets.masked_select(src_masks).float())
        d_loss = self.mse_loss(log_d_predictions.masked_select(src_masks).float(), log_d_targets.masked_select(src_masks).float())
        h2m_loss = self.mae_loss(mel_targets.masked_select(trg_masks), h2m_predictions.masked_select(trg_masks))
        
        total_loss = p_loss + e_loss + d_loss + h2m_loss
    
        return (total_loss, h2m_loss, e_loss, p_loss, d_loss)

### Variance Predictor and Adaptor Start

In [22]:
class Conv(nn.Module):
    """
    Convolution Module
    """

    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size=1,
        stride=1,
        padding=0,
        dilation=1,
        bias=True,
        w_init="linear",
    ):
        """
        :param in_channels: dimension of input
        :param out_channels: dimension of output
        :param kernel_size: size of kernel
        :param stride: size of stride
        :param padding: size of padding
        :param dilation: dilation rate
        :param bias: boolean. if True, bias is included.
        :param w_init: str. weight inits with xavier initialization.
        """
        super(Conv, self).__init__()

        self.conv = nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            bias=bias,
        )

    def forward(self, x):
        x = x.contiguous().transpose(1, 2)
        x = self.conv(x)
        x = x.contiguous().transpose(1, 2)

        return x

In [23]:
class VariancePredictor(nn.Module):
    def __init__(self, input_size, filter_size, kernel, conv_output_size, dropout):
        super(VariancePredictor, self).__init__()
        self.input_size = input_size
        self.filter_size = filter_size
        self.kernel = kernel
        self.conv_output_size = conv_output_size
        self.dropout = dropout
        
        self.conv_layer = nn.Sequential(
            OrderedDict(
                [
                    (
                        "conv1d_1",
                        Conv(
                            self.input_size,
                            self.filter_size,
                            kernel_size=self.kernel,
                            padding=(self.kernel - 1) // 2,
                        ),
                    ),
                    ("relu_1", nn.ReLU()),
                    ("layer_norm_1", nn.LayerNorm(self.filter_size)),
                    ("dropout_1", nn.Dropout(self.dropout)),
                    (
                        "conv1d_2",
                        Conv(
                            self.filter_size,
                            self.filter_size,
                            kernel_size=self.kernel,
                            padding=1,
                        ),
                    ),
                    ("relu_2", nn.ReLU()),
                    ("layer_norm_2", nn.LayerNorm(self.filter_size)),
                    ("dropout_2", nn.Dropout(self.dropout)),
                ]
            )
        )

        self.linear_layer = nn.Linear(self.conv_output_size, 1)

    def forward(self, encoder_output, mask):
        out = self.conv_layer(encoder_output)
        out = self.linear_layer(out)
        out = out.squeeze(-1)

        if mask is not None:
            out = out.masked_fill(mask, 0.0)

        return out

In [29]:
class VarianceAdaptor(nn.Module):
    """Variance Adaptor"""

    def __init__(self, 
                 stats_path,
                 input_size=256,
                 filter_size=256,
                 kernel=3,
                 conv_output_size=256,
                 dropout=0.5,
                 quantization='linear',
                 quantization_n_bins=256,
                 feature_level='phoneme_level'):
        super(VarianceAdaptor, self).__init__()
        self.duration_predictor = VariancePredictor(input_size, filter_size, kernel, conv_output_size, dropout)
        self.length_regulator = LengthRegulator()
        self.pitch_predictor = VariancePredictor(input_size, filter_size, kernel, conv_output_size, dropout)
        self.energy_predictor = VariancePredictor(input_size, filter_size, kernel, conv_output_size, dropout)
        self.pitch_feature_level = feature_level
        self.energy_feature_level = feature_level

        pitch_quantization = quantization
        energy_quantization = quantization
        n_bins = quantization_n_bins
        
        assert pitch_quantization in ["linear", "log"]
        assert energy_quantization in ["linear", "log"]
        
        with open(
            os.path.join(stats_path, "stats.json")
        ) as f:
            stats = json.load(f)
            pitch_min, pitch_max = stats["pitch"][:2]
            energy_min, energy_max = stats["energy"][:2]

        if pitch_quantization == "log":
            self.pitch_bins = nn.Parameter(
                torch.exp(
                    torch.linspace(np.log(pitch_min), np.log(pitch_max), n_bins - 1)
                ),
                requires_grad=False,
            )
        else:
            self.pitch_bins = nn.Parameter(
                torch.linspace(pitch_min, pitch_max, n_bins - 1),
                requires_grad=False,
            )
        if energy_quantization == "log":
            self.energy_bins = nn.Parameter(
                torch.exp(
                    torch.linspace(np.log(energy_min), np.log(energy_max), n_bins - 1)
                ),
                requires_grad=False,
            )
        else:
            self.energy_bins = nn.Parameter(
                torch.linspace(energy_min, energy_max, n_bins - 1),
                requires_grad=False,
            )

        self.pitch_embedding = nn.Embedding(
            n_bins, input_size
        )
        self.energy_embedding = nn.Embedding(
            n_bins, input_size
        )

    def get_pitch_embedding(self, x, target, mask, control):
        prediction = self.pitch_predictor(x, mask)
        if target is not None:
            embedding = self.pitch_embedding(torch.bucketize(target, self.pitch_bins))
        else:
            prediction = prediction * control
            embedding = self.pitch_embedding(
                torch.bucketize(prediction, self.pitch_bins)
            )
        return prediction, embedding

    def get_energy_embedding(self, x, target, mask, control):
        prediction = self.energy_predictor(x, mask)
        if target is not None:
            embedding = self.energy_embedding(torch.bucketize(target, self.energy_bins))
        else:
            prediction = prediction * control
            embedding = self.energy_embedding(
                torch.bucketize(prediction, self.energy_bins)
            )
        return prediction, embedding

    def forward(
        self,
        x,
        src_mask,
        mel_mask=None,
        max_len=None,
        pitch_target=None,
        energy_target=None,
        duration_target=None,
        p_control=1.0,
        e_control=1.0,
        d_control=1.0,
    ):

        log_duration_prediction = self.duration_predictor(x, src_mask)
        
        if self.pitch_feature_level == "phoneme_level":
            pitch_prediction, pitch_embedding = self.get_pitch_embedding(
                x, pitch_target, src_mask, p_control
            )
            x = x + pitch_embedding

        if self.energy_feature_level == "phoneme_level":
            energy_prediction, energy_embedding = self.get_energy_embedding(
                x, energy_target, src_mask, p_control
            )
            x = x + energy_embedding

        if duration_target is not None:
            x = self.length_regulator(x, duration_target)
            duration_rounded = duration_target
        else:
            duration_rounded = torch.clamp(
                (torch.round(torch.exp(log_duration_prediction) - 1) * d_control),
                min=0,
            ).type(torch.LongTensor)
            x = self.length_regulator(x, duration_rounded)

        return (
            x,
            pitch_prediction,
            energy_prediction,
            log_duration_prediction,
            duration_rounded,
        )

In [30]:
class FastSpeech(nn.Module):
    def __init__(self, 
                 stats_path,
                 num_heads=2,
                 num_layers=6,
                 dropout=0.0,
                 embed_dim=256,
                 hidden_dim=512,
                 kernel_size=3,
                 max_trg_seq_len=1000,
                 max_src_seq_len=200,
                 src_vocab_size=360,
                 n_mels=80,
                 input_size=256,
                 filter_size=256,
                 kernel=3,
                 conv_output_size=256,
                 variance_adaptor_dropout=0.5,
                 quantization='linear',
                 quantization_n_bins=256,
                 feature_level='phoneme_level'
                ):
        super().__init__()
        self.encoder_max_seq_len = max_src_seq_len
        self.decoder_max_seq_len = max_trg_seq_len
        self.embed_dim = embed_dim
        self.n_mels = n_mels
        
        self.variance_adaptor = VarianceAdaptor(stats_path, 
                                                input_size=input_size,
                                                filter_size=filter_size,
                                                kernel=kernel,
                                                conv_output_size=conv_output_size,
                                                dropout=variance_adaptor_dropout,
                                                quantization=quantization,
                                                quantization_n_bins=quantization_n_bins,
                                                feature_level=feature_level)
        
        self.encoder = TransformerEncoder(num_heads=num_heads,
                                          num_layers=num_layers,
                                          dropout=dropout,
                                          embed_dim=embed_dim,
                                          hidden_dim=hidden_dim,
                                          kernel_size=kernel_size,
                                          src_vocab_size=src_vocab_size,
                                          max_src_seq_len=max_src_seq_len)
        
        self.decoder = TransformerDecoder(num_heads=num_heads,
                                          num_layers=num_layers,
                                          dropout=dropout,
                                          embed_dim=embed_dim,
                                          hidden_dim=hidden_dim,
                                          kernel_size=kernel_size,
                                          max_trg_seq_len=max_trg_seq_len)
        
        self.hidden2mel = Hidden2Mel(n_mels=n_mels, embed_dim=embed_dim)
        
        self.length_regulator = LengthRegulator(max_trg_seq_len)
        
    def _get_masks(self, seq_lens, max_seq_len, embed_dim):
        B = seq_lens.shape[0]
        masks = [[ mask_idx >= seq_lens[seq_idx]  for mask_idx in torch.arange(max_seq_len)]  for seq_idx in torch.arange(0, B)]
        masks = torch.tensor(masks)
        fill_masks = repeat(masks.T, 'b s -> e b s', e=embed_dim).T.contiguous()
        
        return masks.to(device), fill_masks.to(device)
        
    def forward(self, src_seq, src_seq_len, trg_seq, trg_seq_len, trg_durations, trg_energy, trg_pitch):
        src_masks, src_fill_masks = self._get_masks(src_seq_len, self.encoder_max_seq_len, self.embed_dim)
        trg_masks, trg_fill_masks = self._get_masks(trg_seq_len, self.decoder_max_seq_len, self.embed_dim)
        mel_masks, mel_fill_masks = self._get_masks(trg_seq_len, self.decoder_max_seq_len, self.n_mels)
        
        enc_out = self.encoder(src_seq, src_masks, src_fill_masks)
#         adapted_enc_out = self.length_regulator(enc_out, trg_durations)

        (adapted_enc_out, 
        p_predictions,
        e_predictions,
        log_d_predictions,
        d_rounded) = self.variance_adaptor(
            enc_out,
            src_masks,
            mel_mask=trg_masks,
            duration_target=trg_durations)
        
        dec_out = self.decoder(adapted_enc_out, trg_masks, trg_fill_masks)
        h2m_out = self.hidden2mel(dec_out).masked_fill(mel_fill_masks, 0.0)
        
        return (h2m_out,
                p_predictions,
                e_predictions,
                log_d_predictions,
                d_rounded,
                src_masks,
                mel_fill_masks)
    

# model = FastSpeech(
#     './preprocessed_data/SpanishSingleSpeaker',
#     num_heads=2,
#     num_layers=6,
#     dropout=0.0, 
#     embed_dim=256,
#     hidden_dim=512,
#     kernel_size=3,
#     max_trg_seq_len=1000,
#     max_src_seq_len=200,
#     n_mels=80,
#     src_vocab_size=len(_symbol_to_id)
# )

# phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch = next(iter(train_loader))

# (h2m_predictions,
# p_predictions,
# e_predictions,
# log_d_predictions,
# d_rounded,
# src_masks,
# trg_masks) = model(phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch)

print('Success')

Success


In [31]:
t, m, v, s, d = FastSpeechLoss(200, 1000)(h2m_predictions,
                          mels,
                          p_predictions,
                          pitch,
                          e_predictions,
                          energy,
                          log_d_predictions,
                          durations,
                          src_masks,
                          trg_masks)


NameError: name 'h2m_predictions' is not defined

### Variance Predictor and Adaptor End

In [32]:
class FastSpeechModule(pl.LightningModule):
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = FastSpeech(
#                         './preprocessed_data/SpanishSingleSpeaker',
                        './preprocessed_data/LJSpeech',
                        num_heads=2,
                        num_layers=6,
                        dropout=0.0, 
                        embed_dim=256,
                        hidden_dim=512,
                        kernel_size=3,
                        max_trg_seq_len=1000,
                        max_src_seq_len=200,
                        n_mels=80,
                        src_vocab_size=len(_symbol_to_id)
                    )
        
        self.loss_module = FastSpeechLoss(encoder_max_seq_len=200, decoder_max_seq_len=1000)
        
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
        return [optimizer], [lr_scheduler]
    
    def training_step(self, batch, batch_idx):
        phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch = batch
        
        (
            h2m_predictions,
            p_predictions,
            e_predictions,
            log_d_predictions,
            d_rounded,
            src_masks,
            trg_masks
        ) = self.model(phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch)
        
        
        (
            total_loss,
            h2m_loss,
            e_loss,
            p_loss,
            d_loss) = self.loss_module(
            h2m_predictions,
            mels,
            p_predictions,
            pitch,
            e_predictions,
            energy,
            log_d_predictions,
            durations,
            src_masks,
            trg_masks
        )
        
        self.log(f'train_total_loss', total_loss, on_step=True, on_epoch=True)
        self.log(f'train_h2m_loss', h2m_loss, on_step=True, on_epoch=True)
        self.log(f'train_e_loss', e_loss, on_step=True, on_epoch=True)
        self.log(f'train_d_loss', d_loss, on_step=True, on_epoch=True)
        self.log(f'train_p_loss', p_loss, on_step=True, on_epoch=True)
        
        if self.current_epoch in [100, 150, 198, 298]:
            inv_mel_spec(h2m_predictions[0].T, f'file_{self.current_epoch}.wav', stft)
            print('synth')
        
        return total_loss
    
    def validation_step(self, batch, batch_idx):
        phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch = batch
        
        (
            h2m_predictions,
            p_predictions,
            e_predictions,
            log_d_predictions,
            d_rounded,
            src_masks,
            trg_masks
        ) = self.model(phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch)
        
        
        (
            total_loss,
            h2m_loss,
            e_loss,
            p_loss,
            d_loss) = self.loss_module(
            h2m_predictions,
            mels,
            p_predictions,
            pitch,
            e_predictions,
            energy,
            log_d_predictions,
            durations,
            src_masks,
            trg_masks
        )
        
        self.log(f'val_total_loss', total_loss, on_step=True, on_epoch=True)
        self.log(f'val_h2m_loss', h2m_loss, on_step=True, on_epoch=True)
        self.log(f'val_e_loss', e_loss, on_step=True, on_epoch=True)
        self.log(f'val_d_loss', d_loss, on_step=True, on_epoch=True)
        self.log(f'val_p_loss', p_loss, on_step=True, on_epoch=True)   
    
    def test_step(self, batch, batch_idx):
        phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch = batch
        
        (
            h2m_predictions,
            p_predictions,
            e_predictions,
            log_d_predictions,
            d_rounded,
            src_masks,
            trg_masks
        ) = self.model(phonemes, phoneme_lens, mels, mel_lens, durations, energy, pitch)
        
        (
            total_loss,
            h2m_loss,
            e_loss,
            p_loss,
            d_loss) = self.loss_module(
            h2m_predictions,
            mels,
            p_predictions,
            pitch,
            e_predictions,
            energy,
            log_d_predictions,
            durations,
            src_masks,
            trg_masks
        )
        
        self.log(f'test_total_loss', total_loss, on_step=True, on_epoch=True)
        self.log(f'test_h2m_loss', h2m_loss, on_step=True, on_epoch=True)
        self.log(f'test_e_loss', e_loss, on_step=True, on_epoch=True)
        self.log(f'test_d_loss', d_loss, on_step=True, on_epoch=True)
        self.log(f'test_p_loss', p_loss, on_step=True, on_epoch=True)   

In [33]:
import wandb
wandb.finish()
wandb_logger = WandbLogger(project='Fast Speech 2 - Improved', name='LJ Speech with var adapter 1')

In [34]:
def train_model(**kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join('checkpoints', 'initial model x'),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=300,
                         logger=wandb_logger,
                         overfit_batches=1,
                         callbacks=[
                             ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_total_loss"),
                                    LearningRateMonitor("epoch")]
                        )
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pl.seed_everything(42) # To be reproducable
    model = FastSpeechModule(**kwargs)
    trainer.fit(model, train_loader, val_loader)
    model = FastSpeechModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    result = {"val": val_result[0]["test_loss"]}
    return model, result

model, results = train_model(lr=3e-4)

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

synth


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

KeyError: 'test_loss'

### ViT Stuff

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, kernel_size, dropout=0.0):
        super().__init__()
        
        input_dim = patch_size * patch_size * num_channels
        
        self.input_net = nn.Sequential(
            Rearrange('B C (h p1) (w p2) -> B (h w) (C p1 p2)', p1=patch_size, p2=patch_size),
            nn.Linear(input_dim, embed_dim)
        )
        
        self.positional_embedding = nn.Embedding(num_patches + 1, embed_dim) # +1 for the CLS token
        
        self.mlp = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes),
        )
        
        self.encoder = nn.Sequential(*[
            TransformerBlock(embed_dim, num_heads, dropout, hidden_dim, kernel_size) for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
        
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        
    def forward(self, x):
        x = self.input_net(x)
        B, T, _ = x.shape
        cls_token = repeat(self.cls_token, '1 1 N -> B 1 N', B=B)
        x = torch.cat([cls_token, x], dim=1)
        pos = torch.arange(0, T+1).to(device)
        x = x + self.positional_embedding(pos)
        x = self.dropout(x)
        x = self.encoder(x)
        out = self.mlp(x[:, 0, :])
        return out
    
t = VisionTransformer(**{ 'embed_dim': 256,
                                'hidden_dim': 512,
                                'num_heads': 8,
                                'num_layers': 6,
                                'patch_size': 4,
                                'num_channels': 3,
                                'num_patches': 64,
                                'num_classes': 10,
                                'dropout': 0.2,
                                'kernel_size': 3
                        })

# t(torch.randn(6, 3, 32, 32)).shape

class ViT(pl.LightningModule):

    def __init__(self, model_kwargs, lr):
        super().__init__()
        self.save_hyperparameters()
        self.model = VisionTransformer(**model_kwargs)
        self.example_input_array = next(iter(train_loader))[0]

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode="train"):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(f'{mode}_loss', loss, on_step=True, on_epoch=True)
        self.log(f'{mode}_acc', acc, on_step=True, on_epoch=True)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="test")

In [ ]:
wandb.finish()
wandb_logger = WandbLogger(project='FastSpeech2', name='ViT CNN v2')

In [ ]:
def train_model(**kwargs):
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, "ViT CNN"),
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=180,
                         logger=wandb_logger,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pl.seed_everything(42) # To be reproducable
    model = ViT(**kwargs)
    trainer.fit(model, train_loader, val_loader)
    model = ViT.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}
    return model, result

model, results = train_model(model_kwargs={
                                'embed_dim': 256,
                                'hidden_dim': 512,
                                'num_heads': 8,
                                'num_layers': 6,
                                'patch_size': 4,
                                'num_channels': 3,
                                'num_patches': 64,
                                'num_classes': 10,
                                'dropout': 0.2,
                                'kernel_size': 3
                            },
                            lr=3e-4)

print("ViT results", results)